# Algoritmo de Machine Learning para Previsão de Churn

## Objetivo principal: 
Este é um projeto para a formação __Cientista de Dados__ da [Data Science Academy](https://www.datascienceacademy.com.br), com o objetivo de construir um algoritmo de Machine Learning, demonstrando passo a passo do desenvolvimento.

__Modelo:__ Será construido do zero uma versão simplificada da [Regressão Logistica](https://pt.wikipedia.org/wiki/Regress%C3%A3o_log%C3%ADstica), sendo este um algoritmo aplicado a problemas de classificação. 

As bases principais utilizadas para definir os cálculos das funções do algoritmo são:
- [CS229 Lecture Notes](https://cs229.stanford.edu/lectures-spring2022/main_notes.pdf) por [Andrew Ng](https://www.andrewng.org/).
- [Probability for Machine Learning - Discover How To Harness Uncertainty With Python](https://dokumen.pub/probability-for-machine-learning-discover-how-to-harness-uncertainty-with-python-v19nbsped.html) por Jason Brownlee

- [Machine Learning - A Probabilistic Perspective](http://noiselab.ucsd.edu/ECE228/Murphy_Machine_Learning.pdf) por Kevin P. Murphy

## Objetivo secundário:
Utilizar o algoritmo desenvolvido para solucionar um problema de negócio. O algoritmo construído será utilizado para previsão de churn. 

Os dados estão disponíveis no Kaggle e o objetivo é __identificar os clientes que irão sair ou não nos próximos 6 meses__.

Obs.: Ao longo do desenvolvimento do projeto, podem ser utilizadas bibliotecas e frameworks de mercado disponíveis para realizar etapas de pré-processamento e tratamentos gerais dos dados, mas o algoritmo de classificação e todo seu desenvolvimento serão realizados neste notebook.

## Construindo o algoritmo

<!-- 
https://web.stanford.edu/~jurafsky/slp3/5.pdf

https://medium.com/@msremigio/regress%C3%A3o-log%C3%ADstica-logistic-regression-997c6259ff9a

https://medium.com/turing-talks/turing-talks-14-modelo-de-predi%C3%A7%C3%A3o-regress%C3%A3o-log%C3%ADstica-7b70a9098e43

https://matheusfacure.github.io/2017/02/25/regr-log/

https://www.datacamp.com/tutorial/understanding-logistic-regression-python

https://monografias.ufma.br/jspui/bitstream/123456789/3572/1/LEANDRO-GONZALEZ.pdf -->


A __regressão logística__ pode ser usada para vários problemas de classificação binária, como detecção de spam, previsão de diabetes, se cliente vai comprar um produto ou vai deixar de ser um cliente, etc. Se trata de um __algoritmo linear__, assim como a [regressão linear](https://pt.wikipedia.org/wiki/Regress%C3%A3o_linear), porém suas saídas são probabilidades dos eventos acontecerem, sendo essas probabilidades transformadas em valores geralmente binários (0 ou 1). 

Para construir o algoritmo da regressão logistica será necessário definir parametros de iniciais, uma __função de custo__ e uma __função de otimização__ que irá de for iterativa, atualizar os parametros até a convergencia. O algoritmo usa da __função logística sigmoide__ para modelar a probabilidade dos eventos estimados.

### __Sigmoide__

<!-- https://matheusfacure.github.io/2017/07/12/activ-func/

https://iaexpert.academy/2020/05/25/funcoes-de-ativacao-definicao-caracteristicas-e-quando-usar-cada-uma/ -->

[Sigmoid](https://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_log%C3%ADstica) é uma função matemática que transforma qualquer número real em uma probabilidade entre 0 e 1. Graficamente, a sigmoide representa uma curva em forma de S, indicando que quanto mais `z` se aproxima de infinito positivo, mais o valor previsto se aproxima de 1 e quanto mais `z` se aproximar ao infinito negativo, mais o valor previsto será aproximado de 0.

A função logistica é definida como:

$$\sigma(z)=\frac{1}{1+e^{-z}}$$

Representada graficamente:

<img src="https://ml-cheatsheet.readthedocs.io/en/latest/_images/sigmoid.png" alt="drawing" style="width:250px;"/>

Onde:
- __σ(z)__ = Estimativa de probabilidade (p), sendo uma saída entre 0 e 1
- __z__ = Entrada para a função (estimativa do algoritmo, por exemplo βx+α)
- __e__ = [Constante de Euler](https://pt.wikipedia.org/wiki/E_(constante_matem%C3%A1tica) ), base dos logaritmos naturais

Como __limite de decisão__ para mapear as probabilidades (p) em classes discretas binárias (0 ou 1) consideramos o valor de __0.5__, conforme a regra abaixo:

$$
p ≥ 0.5, classe=1 \\
p < 0.5, classe=0
$$

In [1]:
# Definindo a função sigmoide em python
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### __Função de custo/perda__

As funções de perda são usadas para determinar o custo (também conhecido como "erro") entre a saída do algoritmo `ŷ` e o valor alvo real `y`, ou seja, a função de perda gera uma métrica que indica o quão longe as estimativas geradas estão dos valores reais. Como o objetivo é ser o mais assertivo quanto possível, é necessário reduzir os custos gradativamente e isso é possível ser feito utilizando uma função de otimização.

Apesar de a Regressão Logistica ser um algoritmo linear, não devemos utilizar a mesma função de custo [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) (Mean Squared Error) que usamos para a Regressão Linear. Pulando a parte explicativa matemática¹, de maneira resumida o motivo de não usar a MSE para regressão logistica é que a função de previsão é não linear, devido à transformação sigmóide, e por isso é utilizada uma função de custo mais adequada, a __Entropia Cruzada binária__ ou __Log Loss__, que é dada pela função abaixo:

$$ J(\theta) = -\frac{1}{m}\sum_{n=1}^m[y^{(i)}log(\hat{y}^{(i)})+(1-y^{(i)})log(1-\hat{y}^{(i)})]$$

[1] [Probability for Machine Learning - Discover How To Harness Uncertainty With Python, página 102](https://dokumen.pub/probability-for-machine-learning-discover-how-to-harness-uncertainty-with-python-v19nbsped.html)

In [2]:
# Definindo a função de custo em python
def log_loss(y, yhat):
    m = len(y)
    loss = (-1 / m) * np.sum((y.T.dot(np.log(yhat))) + ((1 - y).T.dot(np.log(1 - yhat))))
    return loss

### Função de otimização

<!-- https://matheusfacure.github.io/2017/02/20/MQO-Gradiente-Descendente/ 
https://developer.ibm.com/articles/implementing-logistic-regression-from-scratch-in-python/
-->

Como o objetivo é minimizar a função de custo, será necessário ajustar os parametros (pesos) até alcançar a convergencia. Esta otimização dos parametros pode ser obtida através da derivada da função de custo em relação a cada peso. Como função de otimização, será aplicado o __gradiente descendente__ (descida gradiente).

Os gradientes são o vetor da derivada de 1ª ordem da função custo. 
Ao diferenciar a função de custo, temos a expressão do gradiente que é dado por:

$$\frac{\partial}{\partial \theta_j} J(\theta) = \frac{1}{m}\sum_{i=1}^m(\hat{y}^{(i)}-y^{(i)})x_j^{(i)}$$

Para ajustar o parâmetro __θ__:

$$ \theta_j^{new}=\theta_j^{old}-\alpha\frac{1}{m}\sum_{i=1}^m(\hat{y}^{(i)}-y^{(i)})x_j^{(i)}$$

Onde __α__ é a taxa de aprendizado (learning rate).

Para mais detalhes sobre a derivação da função de custo da regressão logistica, indico o artigo [The Derivative of Cost Function for Logistic Regression](https://medium.com/analytics-vidhya/derivative-of-log-loss-function-for-logistic-regression-9b832f025c2d)

In [3]:
# Definindo função de otimização em python
def gradient_descent(X, y, theta, yhat, learning_rate):  
    m = len(y)
    theta -= learning_rate * ((1 / m) * (np.dot(X.T, (yhat - y))))
    return theta

---

### Regressão Logistica

Para melhor a aplicação das funções, vou reescreve-las utilizando programação orientada a objetos para criar uma classe que irá representar o algoritmo.

In [4]:
class LogisticRegression:
    """
    Classificador de regressão logística.
    Parametros
    ----------
    n_iterations: int, default=500
        Número máximo de iterações para convergir.
    learning_rate float, default=0.01
        Taxa de aprendizado.
    ----------
    """    
    # Inicializando a função com os parametros learning_rate e n_iterations
    def __init__(self, learning_rate=0.01, n_iterations=500):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
    
    # Implementando a função logistica
    def __sigmoid(self, z):
        np.seterr(all='ignore')
        return 1 / (1 + np.exp(-z))
    
    # Implementando a função de custo: Entropia Cruzada Binária/Log Loss
    def __log_loss(self, y, yhat):        
        return (-1 / self.__m) * np.sum((y.T.dot(np.log(yhat))) + ((1 - y).T.dot(np.log(1 - yhat))))
    
    # Implementando a função de otimização: Gradiente Descendente
    def __gradient_descent(self, X, y, yhat, theta):
        theta -= self.learning_rate * ((1 / self.__m) * (np.dot(X.T, (yhat - y))))
        return theta
    
    # Definindo a função de ajuste do modelo: processo de treinamento
    def fit(self, X, y):
        """
        Ajuste o modelo de acordo com os dados de treinamento fornecidos.
        """
        self.classes_ = np.unique(y)
        self.__m = np.float64(X.shape[0])
        self.loss_lst = list()
        theta = np.zeros((X.shape[1]))
        for _ in range(self.n_iterations):
            z = np.dot(X, theta)            
            yhat = self.__sigmoid(z)
            theta = self.__gradient_descent(X, y, yhat, theta)
            loss = self.__log_loss(y, yhat)
            self.loss_lst.append(loss)
        self.theta = theta        
    
    # Definindo a função de estimador do modelo
    def predict(self, X):
        """
        Prever rótulos de classe para amostras em X.
        """
        z = np.dot(X, self.theta)
        proba = self.__sigmoid(z)
        return np.asarray([1 if p > 0.5 else 0 for p in proba])

O código acima está disponível em [../src/LogisticRegression.py](https://github.com/pedrohrafael/data-science/blob/main/projects/datascienceacademy/churn_prediction/src/LogisticRegression.py)

## Previsão de Churn

<!-- DELETE https://medium.com/data-hackers/churn-prediction-de-uma-empresa-de-telecomunica%C3%A7%C3%B5es-5832b324c5a -->

Churn, representa a taxa de evasão da base de clientes. Churn ou churn rate é uma métrica importante tanto pelo fato de que a perda de clientes leva a uma perda de receita potencial, mas também pelo fato de que o custo para adquirir novos clientes é de um modo geral mais alto que o custo para manter os clientes atuais na base.

Como o objetivo deste projeto é a construção de um algoritmo de machine learning e sua aplicação para solucionar um problema de negócios, as etapas de processamento de dados, validação de métricas e ajuste de hiperparametros serão realizadas o mínimo possível, pois não é o objetivo aqui ter uma alta taxa de assertividade.

Como base de dados, será utilizado o dataset [JOB-A-THON](https://www.kaggle.com/datasets/gauravduttakiit/jobathon-march-2022) disponível no Kaggle.

Este dataset conta com uma base de clientes de um banco, onde o objetivo é prever se os clientes irão sair ou não da base nos próximos 6 meses.

#### Dicionário de dados:

- __ID__ - Identificador exclusivo de uma linha
- __Age__ - Idade do cliente
- __Gender__ - Gênero do cliente (Masculino e Feminino)
- __Income__ - Renda anual do cliente
- __Balance__ - Saldo médio trimestral do cliente
- __Vintage__ - Nº de anos em que o cliente está associado ao banco
- __Transaction_Status__ - Se o cliente fez alguma transação nos últimos 3 meses ou não
- __Product_Holdings__ - Nº de participações de produtos com o banco
- __Credit_Card__ - Se o cliente tem cartão de crédito ou não
- __Credit_Category__ - Categoria de um cliente com base na pontuação de crédito
- __Is_Churn__ - Se o cliente vai sair nos próximos 6 meses ou não

### Importando os dados

In [5]:
import pandas as pd
import numpy as np
import os

In [6]:
ROOT_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
DATA_DIR = os.path.join(ROOT_DIR, "data")

In [7]:
df_labels = pd.read_csv(os.path.join(DATA_DIR, "sample_OoSmYo5.csv"))
df_train = pd.read_csv(os.path.join(DATA_DIR, "train_PDjVQMB.csv"))
df_test = pd.read_csv(os.path.join(DATA_DIR, "test_lTY72QC.csv"))

In [8]:
df_test = pd.merge(df_test, df_labels)

In [9]:
df_train.sample(5)

,ID,Age,Gender,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category,Is_Churn
5897,d1b8f880,60,Female,10L - 15L,912813.66,3,1,2,0,Poor,0
4874,b3ea2c1e,51,Male,5L - 10L,2394558.81,3,1,1,1,Good,0
2441,704c27de,32,Male,5L - 10L,741359.79,1,1,2,1,Poor,0
506,7b03b242,56,Male,5L - 10L,2198194.11,0,1,1,0,Good,1
4553,1886a5fd,51,Female,More than 15L,1044761.31,4,0,1,1,Poor,0


### Explorando os dados

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6650 entries, 0 to 6649
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  6650 non-null   object 
 1   Age                 6650 non-null   int64  
 2   Gender              6650 non-null   object 
 3   Income              6650 non-null   object 
 4   Balance             6650 non-null   float64
 5   Vintage             6650 non-null   int64  
 6   Transaction_Status  6650 non-null   int64  
 7   Product_Holdings    6650 non-null   object 
 8   Credit_Card         6650 non-null   int64  
 9   Credit_Category     6650 non-null   object 
 10  Is_Churn            6650 non-null   int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 571.6+ KB


In [11]:
df_train.describe(percentiles=[.1, .25, .5, .75, .9]).T

,count,mean,std,min,10%,25%,50%,75%,90%,max
Age,6650.0,41.130226,9.685747,21.0,29.000,34.0000,40.000,47.00,55.000,72.00
Balance,6650.0,804595.354985,515754.867315,63.0,129847.401,392264.2125,764938.575,1147123.71,1511154.522,2436615.81
Vintage,6650.0,2.250226,1.458795,0.0,0.000,1.0000,2.000,3.00,4.000,5.00
Transaction_Status,6650.0,0.515789,0.499788,0.0,0.000,0.0000,1.000,1.00,1.000,1.00
Credit_Card,6650.0,0.664361,0.472249,0.0,0.000,0.0000,1.000,1.00,1.000,1.00
Is_Churn,6650.0,0.231128,0.421586,0.0,0.000,0.0000,0.000,0.00,1.000,1.00


In [12]:
(df_train.Is_Churn.value_counts(normalize=True)).to_frame().round(2)

,Is_Churn
0,0.77
1,0.23


A variável alvo  dos dados de treino, __Is_Churn__, está desbalanceada, onde temos mais amostras de clientes "não churn" do que amostras de "churn" que queremos prever.

O dataset conta com variáveis categoricas e numericas. Será necessário aplicar transformações em algumas variáveis antes de treinar o modelo.

### Engenharia de recursos

#### Removendo coluna ID

In [13]:
# Removendo a coluna ID
df_train.drop(columns=['ID'], inplace=True)
df_test.drop(columns=['ID'], inplace=True)

#### Label encoder

##### Variáveis categorias ordinais

In [14]:
# Mapeando os valores numeros das variaveis categóricas ordinais
Income_map = {'Less than 5L':0, '5L - 10L':1, '10L - 15L':2, 'More than 15L': 3}
Credit_Category_map = {'Poor':0, 'Average':1, 'Good':2}
Product_Holdings_map = {'1':0, '2':1, '3+':2}

In [15]:
# Transformando as variaveis categoricas ordinais em numericas
df_train['Income'] = df_train['Income'].map(Income_map)
df_train['Credit_Category'] = df_train['Credit_Category'].map(Credit_Category_map)
df_train['Product_Holdings'] = df_train['Product_Holdings'].map(Product_Holdings_map)

df_test['Income'] = df_test['Income'].map(Income_map)
df_test['Credit_Category'] = df_test['Credit_Category'].map(Credit_Category_map)
df_test['Product_Holdings'] = df_test['Product_Holdings'].map(Product_Holdings_map)

##### Variáveis categorias nominais

In [16]:
# Transformando as variaveis categoricas nominais em numericas
df_train = pd.get_dummies(df_train, prefix='', prefix_sep='', columns=['Gender'])
df_test = pd.get_dummies(df_test, prefix='', prefix_sep='', columns=['Gender'])

#### Dimensionamento de recursos

##### Variáveis numéricas contínuas

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
# Instanciando o scaler
scaler = MinMaxScaler()

# Definindo as variaveis numericas continuas
num_cont_cols = ['Age', 'Balance']

# Treino do dimensionamento das variaveis
scaler.fit(df_train[num_cont_cols])

# Aplicando dimensionamento das variaveis
df_train.loc[:, num_cont_cols] = scaler.transform(df_train[num_cont_cols])
df_test.loc[:, num_cont_cols] = scaler.transform(df_test[num_cont_cols])

In [28]:
df_train.sample(5)

,Age,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category,Is_Churn,Female,Male
4393,0.529412,1,0.315408,4,0,2,1,2,1,0,1
4759,0.725490,2,0.583565,3,1,1,1,2,1,1,0
561,0.568627,1,0.888172,3,0,0,1,0,0,0,1
4494,0.078431,0,0.476183,3,1,0,0,1,0,1,0
1609,0.470588,1,0.027034,3,1,0,1,1,0,0,1


#### Reamostragem

In [19]:
# Criando arrays das variáveis dependentes e da variavel alvo
X_train = np.asarray(df_train.drop(columns=['Is_Churn']))
y_train = np.asarray(df_train.Is_Churn)

X_test = np.asarray(df_test.drop(columns=['Is_Churn']))
y_test = np.asarray(df_test.Is_Churn)

In [20]:
from imblearn.under_sampling import RandomUnderSampler

In [21]:
# Instanciando a classe de sub-amostragem
undersample = RandomUnderSampler(sampling_strategy='majority')

# Aplicando sub-amostragem da classe majoritária (0 : 'não churn')
X_train, y_train = undersample.fit_resample(X_train, y_train)

### Treinando o modelo

In [22]:
# Importando metricas de avaliação do modelo
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [26]:
# Instanciando o modelo criado
my_lr = LogisticRegression(learning_rate=0.01, n_iterations=1000)

# Realizando o treinamento
my_lr.fit(X_train, y_train)

# Salvando as estimativas geradas
my_ypred_train = my_lr.predict(X_train)
my_ypred_test = my_lr.predict(X_test)

# Print das métricas de avaliação: Acurácia e F1 Score
print("Treino")
print("accuracy_score:", accuracy_score(y_train, my_ypred_train).round(2))
print("f1_score:", f1_score(y_train, my_ypred_train, average='macro').round(2))

print("Teste")
# Print das métricas de avaliação: Acurácia e F1 Score
print("accuracy_score:", accuracy_score(y_test, my_ypred_test).round(2))
print("f1_score:", f1_score(y_test, my_ypred_test, average='macro').round(2))

Treino
accuracy_score: 0.57
f1_score: 0.57
Teste
accuracy_score: 0.46
f1_score: 0.31


Apesar de as metricas de avaliação das estimativas do modelo não serem muito altas, o exercício serviu para demonstrar o passo a passo da criação do algoritmo de classificação __Regressão Logistica__ e como é possível aplicá-lo para realizar a previsão de Churn.

#### Comparando o modelo com a Regressão Logistica do scikit learn

In [24]:
from sklearn.linear_model import LogisticRegression as sk_LogisticRegression

In [27]:
# Instanciando o modelo sk-learn
sk_lr = sk_LogisticRegression(max_iter=1000)

# Realizando o treinamento
sk_lr.fit(X_train, y_train)

# Salvando as estimativas geradas
sk_ypred_train = sk_lr.predict(X_train)
sk_ypred_test = sk_lr.predict(X_test)

# Print das métricas de avaliação: Acurácia e F1 Score
print("Treino")
print("accuracy_score:", accuracy_score(y_train, sk_ypred_train).round(2))
print("f1_score:", f1_score(y_train, sk_ypred_train, average='macro').round(2))

print("Teste")
# Print das métricas de avaliação: Acurácia e F1 Score
print("accuracy_score:", accuracy_score(y_test, sk_ypred_test).round(2))
print("f1_score:", f1_score(y_test, sk_ypred_test, average='macro').round(2))

Treino
accuracy_score: 0.61
f1_score: 0.61
Teste
accuracy_score: 0.42
f1_score: 0.29


Podemos ver que o modelo criado tem as métricas de avaliação bem próximas do modelo de Regressão Logistica do scikit learn.